In [1]:
#pip install dash
import wget

In [2]:
#get the SpaceX Launch dataset for this lab
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
wget.download(url)

'spacex_launch_dash (5).csv'

In [3]:
#Download a skeleton Dash app to be completed in this lab
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/spacex_dash_app.py'
wget.download(url)

'spacex_dash_app (3).py'

In [4]:
#import warnings
#warnings.filterwarnings("ignore")
#Test the skeleton app by running the following command in the terminal:
#%run spacex_dash_app.py

In [5]:
# Import required libraries
import pandas as pd
import dash
from dash import html
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [6]:
# Read the data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [7]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [8]:
ls = list(spacex_df['Launch Site'].unique())

In [9]:
ls.sort()
ls


['CCAFS LC-40', 'CCAFS SLC-40', 'KSC LC-39A', 'VAFB SLC-4E']

In [10]:
max_payload, min_payload

(9600.0, 0.0)

In [18]:
# Create a dash application
app = dash.Dash(__name__)

#list of Launch Sites
launch_sites = list(spacex_df['Launch Site'].unique())
launch_sites.sort()

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                # dcc.Dropdown(id='site-dropdown',...)
                                dcc.Dropdown(id='site-dropdown', 
                                            options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'},
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}
                                            ],
                                             value = 'ALL',  
                                             placeholder="Select a launch site here",
                                             searchable = True,
                                             style={'width':'80%', 'padding':'3px', 'font-size': '15px', 'text-align-last' : 'center'},
                                            ),  
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                
                                # TASK 3: Add a slider to select payload range
                                #dcc.RangeSlider(id='payload-slider',...)
                                dcc.RangeSlider(min = 0, max = 10000, step = 1000, value = [0, 10000], id='payload-slider'),
 

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

# Function decorator to specify function input and output 
@app.callback(
                Output(component_id='success-pie-chart', component_property='figure'),
                Input(component_id='site-dropdown', component_property='value')
             ) 

def get_pie_chart(entered_site):
    filtered_df = spacex_df[['Launch Site', 'class']]
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values=filtered_df['class'], names=filtered_df['Launch Site'], title='Total Success Launch By Site')
    else:
    # return the outcomes piechart for a selected site
        data = filtered_df[filtered_df ['Launch Site'] == entered_site].value_counts()
        fig = px.pie(data, values=data, names=['Failure', 'Success'], title='Total Success Launch for site '+ entered_site)
    return fig


# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    Input(component_id='site-dropdown', component_property='value'),
    Input(component_id='payload-slider', component_property='value')  
) 

def get_success_payload_scatter_chart(entered_site, slider_range) :
    min, max = slider_range
    dat = spacex_df[(spacex_df['Payload Mass (kg)']>= min) & (spacex_df['Payload Mass (kg)']<= max) ]
    if entered_site == 'ALL':
        fig = px.scatter(dat , y = 'class', x = 'Payload Mass (kg)', 
                         color = 'Booster Version Category', 
                         title='Correlation between Payload and Success for all Sites')
    else:
        Data = dat[dat ['Launch Site'] == entered_site]
        fig = px.scatter(Data, y = 'class', x = 'Payload Mass (kg)', 
                         color = 'Booster Version Category', 
                         title='Correlation between Payload and Success for the site '+entered_site)
    return fig


# Run the app
if __name__ == '__main__':
    app.run_server()